In [1]:
import os
import uuid
import shutil
#import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
#from pylatex.section import Chapter, Subsection
#from pylatex import Document, Section, Figure, NoEscape, Command

%run pipeline.ipynb

import warnings
warnings.filterwarnings("ignore")

In [2]:
bestlcs=pd.read_csv('makingtable/bestlcs.csv')

In [3]:
bestlcs

,pop_id,TIC,gaiadr3_source_id,epic_id,per,group,age_Myr,disco_paper,author,paper_year,...,Campaign 6,Campaign 7,Campaign 8,Campaign 9,Campaign 10,Campaign 11,Campaign 12,Campaign 13,Campaign 14,Campaign 15
0,0,59129133.0,3.392549e+18,246676629.0,0.625300,Taurus,2.0,Stauffer_2018,Stauffer,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EPIC-246676629-13-EVEREST-1800,NaN,NaN
1,1,59091144.0,3.392554e+18,246682490.0,3.632400,Taurus,2.0,Stauffer_2018,Stauffer,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EPIC-246682490-13-EVEREST-1800,NaN,NaN
2,2,118769116.0,3.411342e+18,247343526.0,0.356800,Taurus,2.0,Stauffer_2018,Stauffer,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EPIC-247343526-13-EVEREST-1800,NaN,NaN
3,3,175594111.0,6.045819e+18,203354381.0,0.599300,Upper Scorpius,8.0,Stauffer_2018,Stauffer,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,322928171.0,6.048613e+18,203636498.0,0.779400,Upper Scorpius,8.0,Stauffer_2018,Stauffer,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,203,301676454.0,3.795299e+18,NaN,0.382500,Argus,45.0,Bouma_2024,Bouma,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,204,58084670.0,2.034818e+18,NaN,0.465000,Field,NaN,Bouma_2024,Bouma,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,205,67745212.0,3.191995e+18,NaN,0.213333,Columba,42.0,Bouma_2024,Bouma,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,206,5714469.0,3.493814e+18,NaN,0.431250,UCL,16.0,Bouma_2024,Bouma,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
lclist=np.array([], dtype=object)

In [5]:
start_col=bestlcs.columns.get_loc('Sector 1')

In [6]:
for index, row in bestlcs.iterrows(): #iterate through each object
    for col in bestlcs.columns[start_col:]: #iterate through all the columns starting with the sectors
        value=row[col]
        if not pd.isna(value):
            if 'FFI' in value:
                lclist=np.append(lclist, [value])

In [7]:
lclist

array(['TIC-59129133-5-FFI-30min', 'TIC-59129133-32-FFI-10min',
       'TIC-59129133-71-FFI-200s', 'TIC-59091144-5-FFI-30min',
       'TIC-59091144-32-FFI-10min', 'TIC-59091144-71-FFI-200s',
       'TIC-118769116-70-FFI-200s', 'TIC-118769116-71-FFI-200s',
       'TIC-48908525-91-FFI-200s', 'TIC-280945693-10-FFI-30min',
       'TIC-296790810-10-FFI-30min', 'TIC-296790810-11-FFI-30min',
       'TIC-296790810-37-FFI-10min', 'TIC-296790810-38-FFI-10min',
       'TIC-290889135-10-FFI-30min', 'TIC-290889135-11-FFI-30min',
       'TIC-290889135-37-FFI-10min', 'TIC-290889135-38-FFI-10min',
       'TIC-290889135-64-FFI-200s', 'TIC-290889135-65-FFI-200s',
       'TIC-301432612-10-FFI-30min', 'TIC-301432612-11-FFI-30min',
       'TIC-301432612-37-FFI-10min', 'TIC-301432612-64-FFI-200s',
       'TIC-93763678-10-FFI-30min', 'TIC-93763678-64-FFI-200s',
       'TIC-93763678-90-FFI-200s', 'TIC-448002486-11-FFI-30min',
       'TIC-448002486-38-FFI-10min', 'TIC-328468855-11-FFI-30min',
       'TIC-32846

In [18]:
for curstring in lclist:
    get_lc_string(curstring)